In [ ]:
import os, json, pickle
from os.path import join as ojoin

In [ ]:
graph_dir = ojoin('..', '..', '..', '..', 'code', 'fairim', 'networks')
outdir_template = ojoin('..', 'simulation_results', '%s', 'group_maximin')

In [ ]:
def extract_subgroup_influence(graph, attribute, value, influences):
    subgroup_indices = [nodeid for nodeid, attr_dict in graph.nodes(data=True)
                        if attr_dict[attribute] == value]
    subgroup_influence = sum([influences[i] for i in subgroup_indices])
    return subgroup_influence

In [ ]:
def extract_maximin(graph_number, attribute, values, sim_outfile):
    graph_filename = ojoin(graph_dir, 'graph_spa_500_%d.pickle'%graph_number)
    with open(graph_filename, 'rb') as f:
        graph = pickle.load(f)    

    influences = []
    with open(sim_outfile) as f:
        influences = [float(i) for i in f.readlines() if i.strip()]
    
    subgroup_influences = [extract_subgroup_influence(graph, attribute, value, influences) 
                           for value in values]
    return min(subgroup_influences)

In [ ]:
with open('attribute_values.json') as f:
    attribute_values = json.load(f)

In [ ]:
maximin_info = []
for method in ('mip', 'fairim'):
    for graph_number in range(20):
        for attribute in ['ethnicity', 'age', 'gender']:
            sim_outfile = ojoin(outdir_template%method, attribute, 'output_%d.txt'%graph_number)
            values = attribute_values[attribute]
            maximin = extract_maximin(graph_number, attribute, attribute_values[attribute], sim_outfile)
            maximin_dict = dict(graph_name='graph_spa_500_%d'%graph_number,
                                method=method,
                                attribute=attribute,
                                maximin=maximin)
            maximin_info.append(maximin_dict)

In [ ]:
with open('maximin_values.json', 'w') as f:
    json.dump(maximin_info, f)